In [8]:
import numpy as np
from skimage import io
from skimage.util import random_noise
from skimage.metrics import peak_signal_noise_ratio
from matplotlib import pyplot as plt
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
from skimage.transform import resize
from numba import jit, float32

In [9]:
noisy_img = io.imread ("/Users/jiyoung/Desktop/univ/'23-2/Engineering Design/workspace/dog_noisy.png")
clean_img = io.imread ("/Users/jiyoung/Desktop/univ/'23-2/Engineering Design/workspace/dog.png")

In [10]:
# # Gaussian Filter
# def gaussian_kernel(k_size, sigma):
#     size = k_size//2
#     y, x = np.ogrid[-size:size+1, -size:size+1]
#     filter = 1/(2*np.pi * (sigma**2)) * np.exp(-1 *(x**2 + y**2) /(2*(sigma**2)))
#     sum = filter.sum()
#     filter /= sum
#     return filter

@jit(nopython=True)
def gaussian_kernel(k_size, sigma):
    size = k_size // 2
    x, y = np.meshgrid(np.arange(-size, size+1), np.arange(-size, size+1))
    filter = np.exp(-(x**2 + y**2) / (2 * sigma**2)) / (2 * np.pi * sigma**2)
    filter /= np.sum(filter)
    return filter


In [12]:
# Median Filter
def median_filter(noisy_img, filter_size=(3, 3), stride=1):
    img_shape = np.shape(noisy_img)
    result_shape = tuple(np.int64((np.array(img_shape[:2]) - np.array(filter_size)) / stride) + 1) + (img_shape[2],)
    result = np.zeros(result_shape)

    for h in range(0, result_shape[0], stride):
        for w in range(0, result_shape[1], stride):
            for c in range(img_shape[2]):
                tmp = noisy_img[h:h + filter_size[0], w:w + filter_size[1], c].ravel()
                tmp = np.sort(tmp)
                result[h, w, c] = tmp[int(len(tmp) / 2)]

    return result

# median_filtered_image = median_filter(noisy_img)
# # #plt.imsave('median_filtered_image.png', median_filtered_image)
# plt.imshow(median_filtered_image)

median_filtered_img = median_filter(bilateral_median_img)
median_img_resized = resize ( median_filtered_img, clean_img.shape, mode = 'constant', anti_aliasing=True )
print("Median Filter PSNR: ", peak_signal_noise_ratio(clean_img, median_img_resized))

Median Filter PSNR:  24.232556569383526


/var/folders/n1/t3hm6b612gg69mlfjrdv4ht00000gn/T/ipykernel_226/875877441.py:22: UserWarning: Inputs have mismatched dtype.  Setting data_range based on image_true.
  print("Median Filter PSNR: ", peak_signal_noise_ratio(clean_img, median_img_resized))


In [31]:
def bilateral_filter_optimized(noisy_img, k_size=5, sigma_space=4, sigma_intensity=0.5):
    h, w, ch = noisy_img.shape
    bilateral_noisy_img = np.zeros((h, w, ch))

    # Precompute spatial filter
    spatial_filter = gaussian_kernel(k_size, sigma_space)

    for c in range(ch):
        for i in range(h):
            for j in range(w):
                intensity_center = noisy_img[i, j, c]

                weighted_sum = 0.0
                normalization_factor = 0.0

                # Cache indices outside the inner loop
                i_range = range(max(0, i - k_size//2), min(h, i + k_size//2 + 1))
                j_range = range(max(0, j - k_size//2), min(w, j + k_size//2 + 1))

                for m in range(-k_size//2, k_size//2 + 1):
                    for n in range(-k_size//2, k_size//2 + 1):
                        i_neighbor = i + m
                        j_neighbor = j + n
                        
                        if i_neighbor in i_range and j_neighbor in j_range:
                            intensity_neighbor = noisy_img[i_neighbor, j_neighbor, c]
                            
                            # 강도 차이에 대한 가중치
                            weight_intensity = np.exp(-(intensity_center - intensity_neighbor)**2 / (2 * sigma_intensity**2))
                            
                            # 공간적인 가우시안 필터 적용
                            weight_spatial = spatial_filter[m + k_size//2, n + k_size//2]
                            
                            # 각 픽셀에 대한 가중치를 누적
                            weight = weight_intensity * weight_spatial
                            weighted_sum += intensity_neighbor * weight
                            normalization_factor += weight
                
                # 정규화된 결과를 저장
                bilateral_noisy_img[i, j, c] = weighted_sum / normalization_factor
    
    return bilateral_noisy_img

bilateral_median_img = bilateral_filter_optimized(noisy_img)
print("PSNR: ", peak_signal_noise_ratio(clean_img, bilateral_median_img))

/var/folders/n1/t3hm6b612gg69mlfjrdv4ht00000gn/T/ipykernel_99499/895788856.py:29: RuntimeWarning: overflow encountered in ubyte_scalars
  weight_intensity = np.exp(-(intensity_center - intensity_neighbor)**2 / (2 * sigma_intensity**2))


In [4]:
# Median Filter
def median_filter(noisy_img, filter_size=(3, 3), stride=1):
    img_shape = np.shape(noisy_img)
    result_shape = tuple(np.int64((np.array(img_shape[:2]) - np.array(filter_size)) / stride) + 1) + (img_shape[2],)
    result = np.zeros(result_shape)

    for h in range(0, result_shape[0], stride):
        for w in range(0, result_shape[1], stride):
            for c in range(img_shape[2]):
                tmp = noisy_img[h:h + filter_size[0], w:w + filter_size[1], c].ravel()
                tmp = np.sort(tmp)
                result[h, w, c] = tmp[int(len(tmp) / 2)]

    return result

# median_filtered_image = median_filter(noisy_img)
# # #plt.imsave('median_filtered_image.png', median_filtered_image)
# plt.imshow(median_filtered_image)

median_filtered_img = median_filter(bilateral_median_img)
median_img_resized = resize ( median_filtered_img, clean_img.shape, mode = 'constant', anti_aliasing=True )
print("Median Filter PSNR: ", peak_signal_noise_ratio(clean_img, median_img_resized))

Median Filter PSNR:  24.238725052448054


/var/folders/n1/t3hm6b612gg69mlfjrdv4ht00000gn/T/ipykernel_99499/875877441.py:22: UserWarning: Inputs have mismatched dtype.  Setting data_range based on image_true.
  print("Median Filter PSNR: ", peak_signal_noise_ratio(clean_img, median_img_resized))


In [11]:
# Bilateral Filter with Vectorization and Numba JIT
@jit(nopython=True)
def bilateral_filter_optimized(noisy_img, k_size=5, sigma_space=4, sigma_intensity=0.5):
    h, w, ch = noisy_img.shape
    bilateral_noisy_img = np.zeros((h, w, ch))

    # Precompute spatial filter
    spatial_filter = np.zeros((k_size, k_size))
    for m in range(-k_size//2, k_size//2 + 1):
        for n in range(-k_size//2, k_size//2 + 1):
            spatial_filter[m + k_size//2, n + k_size//2] = np.exp(-(m**2 + n**2) / (2 * sigma_space**2))

    for c in range(ch):
        for i in range(h):
            for j in range(w):
                intensity_center = noisy_img[i, j, c]

                weighted_sum = 0.0
                normalization_factor = 0.0

                for m in range(-k_size//2, k_size//2 + 1):
                    for n in range(-k_size//2, k_size//2 + 1):
                        i_neighbor = i + m
                        j_neighbor = j + n

                        if 0 <= i_neighbor < h and 0 <= j_neighbor < w:
                            intensity_neighbor = noisy_img[i_neighbor, j_neighbor, c]

                            # 강도 차이에 대한 가중치
                            weight_intensity = np.exp(-(intensity_center - intensity_neighbor)**2 / (2 * sigma_intensity**2))

                            # 공간적인 가우시안 필터 적용
                            weight_spatial = spatial_filter[m + k_size//2, n + k_size//2]

                            # 각 픽셀에 대한 가중치를 누적
                            weight = weight_intensity * weight_spatial
                            weighted_sum += intensity_neighbor * weight
                            normalization_factor += weight

                # 정규화된 결과를 저장
                bilateral_noisy_img[i, j, c] = weighted_sum / normalization_factor

    return bilateral_noisy_img

bilateral_median_img = bilateral_filter_optimized(noisy_img)
print("PSNR: ", peak_signal_noise_ratio(clean_img, bilateral_median_img))

PSNR:  15.480750332985275


/var/folders/n1/t3hm6b612gg69mlfjrdv4ht00000gn/T/ipykernel_226/2492565275.py:46: UserWarning: Inputs have mismatched dtype.  Setting data_range based on image_true.
  print("PSNR: ", peak_signal_noise_ratio(clean_img, bilateral_median_img))
